Imports

In [49]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from datetime import datetime as dt

Declarations

In [50]:
MAIN_SITE = "https://www.hltv.org"
startPos=0
endPos=5700
offset=100
dates = []
team1 = []
team2 = []
win_team = []
results_loser = []
results_winner = []
event = []
type_of_game = []
link_to_game = []


In [51]:
#For loop for each file scraped 
for file_number in range(startPos,endPos+offset,offset):
    #for each file make BeautifulSoup
    with open(f"HTML/{file_number}.html", "r") as reader:
        html_string = reader.read()
        soup = BeautifulSoup(html_string, 'html.parser')
        # For each day in the file
    for day in soup.find_all(class_="results-sublist"):
        # Get all matches links
        allLinks = day.find_all('a')        
        for link in allLinks:
            if link.has_key('href'):
                href = link['href']
                if r'/matches/' in href:
                    link_to_game.append(MAIN_SITE+href)
        #set variable date for this whole box
        date=day.find_all("div",class_="standard-headline")[0].text
        #For each game in the day
        for game in day.find_all(class_="result-con"): # Each Match
            dates.append(date) # Date
            team1.append(game.find_all(class_="team")[0].text) # Team 1
            if game.find_all(class_="score-won"): #If winner exists
                results_winner.append(game.find_all(class_="score-won")[0].text) # Result Winning Team
                results_loser.append(game.find_all(class_="score-lost")[0].text) # Result Losing Team
            else: #If winner doesn't exist append tie score instead
                results_winner.append(game.find_all(class_="score-tie")[0].text) # Result Winning Team
                results_loser.append(game.find_all(class_="score-tie")[0].text) # Result Losing Team
            team2.append(game.find_all(class_="team")[0+1].text) # Team 2
            if game.find_all(class_="team team-won"):#If winner exists
                win_team.append(game.find_all(class_="team team-won")[0].text) # Winning Team
            else: #If winner doesn't exist append "Tie" instead
                win_team.append("Tie")
            event.append(game.find_all(class_="event")[0].text) # Event
            type_of_game.append(game.find_all(class_="star-cell")[0].text) # Type of Game

KeyboardInterrupt: 

In [ ]:
# results_data = pd.DataFrame(data={"Date":dates,"Team 1":team1,"Team 2":team2,"Winner":win_team,"Score Loser":results_loser,"Score Winner":results_winner,"Event":event,"Best of":type_of_game,"Link":link_to_game})
# results_data

In [ ]:
# results_datefixed=results_data.copy()
# results_datefixed['Date'] = results_data['Date'].str.replace('Results for ', "")
# results_datefixed['Date'] = pd.to_datetime(results_datefixed['Date'])
# results_datefixed['Date']

In [ ]:
# path = "C:\\Users\Djoze\\Documents\\Projects\\CS-GO Statistics\\HTMLGAMES\\"
# dir_list = os.listdir(path)
# dir_list

# for i, game_link in enumerate(link_to_game):
#     file_name = path+f"{[dates[i]]}{team1[i]}-{team2[i]}.html"
#     new_file_name =path+f"{len(link_to_game)-i}-{dates[i]}-{team1[i]}-{team2[i]}.html"
#     if os.path.isfile(file_name):
        
#         os.rename(file_name, new_file_name)

In [105]:
calc = 5800-5799

print(link_to_game[calc])

https://www.hltv.org/matches/2359683/victorum-vs-villainous-esl-challenger-league-season-42-north-america-relegation


In [ ]:
driver_path = "C:\\Users\\Djoze\\Desktop\\chromedriver.exe"
wd = webdriver.Chrome(executable_path=driver_path)
for i, game_link in enumerate(link_to_game):
    file_name = f"HTMLGAMES/{i}-{dates[i]}-{team1[i]}-{team2[i]}.html"
    wd.get(link_to_game[i])
    time.sleep(5)
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(wd.page_source)

C:\Users\Djoze\AppData\Local\Temp\ipykernel_15600\2199323510.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=driver_path)


KeyboardInterrupt: 

In [172]:
maps = []
overtime = []
team1_world_rank = []
team1_rounds = []
team1_rounds_ct = []
team1_rounds_t = []
team1_rating = []
team2_world_rank = []
team2_rounds = []
team2_rounds_ct = []
team2_rounds_t = []
# team2_rounds_ct_percent = []
# team2_rounds_t_percent = []
# team1_rounds_ct_percent = []
# team1_rounds_t_percent = []
team2_rating = []
ranks = []
scores = []
results_ct = []
results_t = []

In [173]:
last_id = 5800
GAME_FILES = ["","",""]
for j, links in enumerate(GAME_FILES):
    file_of_game = f"{last_id-j}-{dates[j]}-{team1[j]}-{team2[j]}.html"
    number_of_maps = 0
    ranks = []
    scores = []
    results_ct = []
    results_t = []

    with open(f"HTMLGAMES/{file_of_game}", "r", encoding="utf-8") as reader:
        html_string = reader.read()
        soup = BeautifulSoup(html_string, 'html.parser')

  
    for rank in soup.find_all(class_= "teamRanking"):
        ranks.append(rank.text.split("#")[-1])

    for flex in soup.find_all(class_="mapholder"):
        for map_played in flex.find_all(class_="mapname"):
            maps.append(map_played.text)
            number_of_maps+=1
            team1_world_rank.append(ranks[0])
            team2_world_rank.append(ranks[1])

    for result_ct in soup.find_all(class_="ct"):
        results_ct.append(result_ct.text)

    for score in soup.find_all(class_="results-team-score"):
        scores.append(score.text)

    for result_t in soup.find_all(class_="t"):
        results_t.append(result_t.text)

    try:
        for i in range(0,number_of_maps*2,2):
            team1_rounds.append(scores[i])
            team2_rounds.append(scores[i+1])
            team1_rounds_ct.append(results_ct[i])
            team2_rounds_ct.append(results_ct[i+1])
            team1_rounds_t.append(results_t[i+1])
            team2_rounds_t.append(results_t[i])
    except IndexError:
                break

    for i in range(number_of_maps):
        if  scores[i] == "-" or scores[i+3] == "-":
                overtime.append("Not Played")
                team1_rounds_ct.append("-")
                team1_rounds_t.append("-")
                team2_rounds_ct.append("-")
                team2_rounds_t.append("-")
        else:
            if int(scores[i])+int(scores[i+3]) > 30:
                overtime.append("Yes")
            else:
                overtime.append("No")
                
    # for i, r in enumerate(team1_rounds_ct):
    #     team1_rounds_ct_percent.append(int(team1_rounds_ct[i]) * 100 / (int(team1_rounds_ct[i])+int(team2_rounds_ct[i])))
    # for i, r in enumerate(team2_rounds_ct):
    #     team2_rounds_ct_percent.append(int(team2_rounds_ct[i]) * 100 / (int(team2_rounds_ct[i])+int(team1_rounds_ct[i])))
    # for i, r in enumerate(team1_rounds_t):
    #     team1_rounds_t_percent.append(int(team1_rounds_t[i]) * 100 / (int(team1_rounds_t[i])+int(team2_rounds_t[i])))
    # for i, r in enumerate(team1_rounds_t):
    #     team2_rounds_t_percent.append(int(team2_rounds_t[i]) * 100 / (int(team2_rounds_t[i])+int(team1_rounds_t[i])))


In [154]:
print(team1_rounds)
print(team2_rounds)
print(scores)


['6', '22', '19']
['16', '20', '17']
['6', '16', '22', '20', '19', '17']


In [174]:
print(file_of_game)
print(maps)
print(number_of_maps)
print(overtime)
print(team1_world_rank)
print(team2_world_rank)
print("score")
print(team1_rounds)
print(team2_rounds)
print("1half")
print(team1_rounds_ct)
print(team2_rounds_t)
print("2half")
print(team1_rounds_t)
print(team2_rounds_ct)

5800-Results for October 18th 2022-Antic-Warriors.html
['Inferno', 'Overpass', 'Dust2']
3
[]
['206', '206', '206']
['226', '226', '226']
score
['16', '16', '-']
['4', '11', '-']
1half
['11', '6']
['4', '9']
2half
['5', '5']
['0', '7']


In [ ]:
results_data_matches = pd.DataFrame(data={"Map":maps,"Overtime":overtime,"Team 1 World Rank":team1_world_rank,"Team 1 Rounds":team1_rounds,"Team 1 Rounds CT":team1_rounds_ct,"Team 1 Rounds T":team1_rounds_t,"Team 2 World Rank":team2_world_rank,"Team 2 Rounds":team2_rounds,"Team 2 Rounds CT":team2_rounds_ct,"Team 2 Rounds T":team2_rounds_t})
results_data_matches

ValueError: All arrays must be of the same length